In [ ]:
from nba_api.stats.static import teams

In [ ]:
nba_teams = teams.get_teams()

In [ ]:
cavs = [team for team in nba_teams if team['abbreviation'] == 'CLE']

In [ ]:
cavs = cavs[0]

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder

In [ ]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=cavs['id'])

In [ ]:
gamefinder

In [ ]:
games_dict = gamefinder.get_normalized_dict()['LeagueGameFinderResults']

In [ ]:
lastGame = games_dict[0]

In [ ]:
from nba_api.stats.endpoints import playbyplay

In [ ]:
pbp = playbyplay.PlayByPlay(lastGame['GAME_ID']).get_data_frames()[0]

In [ ]:
pbp.head()

In [ ]:
pbp.loc[pbp['EVENTMSGTYPE'] == 5].head()

In [ ]:
eventMsgDescriptionToCode = {
    'FIELD_GOAL_MADE':1,
    'FIELD_GOAL_MISSED':2,
    'FREE_THROW_ATTEMPT':3,
    'REBOUND':4,
    'TURNOVER':5,
    'FOUL':6,
    'VIOLATION':7,
    'SUBSTITUTION':8,
    'TIMEOUT':9,
    'JUMP_BALL':10,
    'EJECTION':11,
    'PERIOD_BEGIN':12,
    'PERIOD_END':13,
    'INSTANT_REPLAY':18
}

codeToEventMsgDescription = dict(zip(eventMsgDescriptionToCode.values(), eventMsgDescriptionToCode.keys()))

In [47]:
class DescribeActions:
    def team_of_action(row):
        ret = ""
        if row['HOMEDESCRIPTION'] != None:
            ret = ret + 'Home,'
        if row['NEUTRALDESCRIPTION'] != None:
            ret = ret + "Neutral,"
        if row['VISITORDESCRIPTION'] != None:
            ret = ret + "Visitor,"
        return ret
    def get_action_type(row):
        eventMsgDescriptionToCode = {
            'FIELD_GOAL_MADE':1,
            'FIELD_GOAL_MISSED':2,
            'FREE_THROW_ATTEMPT':3,
            'REBOUND':4,
            'TURNOVER':5,
            'FOUL':6,
            'VIOLATION':7,
            'SUBSTITUTION':8,
            'TIMEOUT':9,
            'JUMP_BALL':10,
            'EJECTION':11,
            'PERIOD_BEGIN':12,
            'PERIOD_END':13,
            'INSTANT_REPLAY':18
        }
        codeToEventMsgDescription = dict(zip(eventMsgDescriptionToCode.values(), eventMsgDescriptionToCode.keys()))

        return codeToEventMsgDescription[row['EVENTMSGTYPE']]
        
    def parse_turnover(row):
        if "Turnover" in str(row['HOMEDESCRIPTION']):
            return "HOME_TURNOVER"
        elif "Turnover" in str(row['VISITORDESCRIPTION']):
            return "VISITOR_TURNOVER"

    def parse(row):
        if DescribeActions.get_action_type(row) == "TURNOVER":
            return DescribeActions.parse_turnover(row)
        else:
            return DescribeActions.team_of_action(row)

In [49]:
for index, row in pbp.iterrows():
    print("Index: " + str(index) + ", Action: " + str(codeToEventMsgDescription[row['EVENTMSGTYPE']]) + ", Team: " + DescribeActions.parse(row))

Index: 0, Action: PERIOD_BEGIN, Team: Neutral,
Index: 1, Action: JUMP_BALL, Team: Home,
Index: 2, Action: FIELD_GOAL_MISSED, Team: Home,
Index: 3, Action: REBOUND, Team: Home,
Index: 4, Action: FIELD_GOAL_MADE, Team: Home,
Index: 5, Action: FIELD_GOAL_MISSED, Team: Visitor,
Index: 6, Action: REBOUND, Team: Home,
Index: 7, Action: FIELD_GOAL_MADE, Team: Home,
Index: 8, Action: FIELD_GOAL_MISSED, Team: Visitor,
Index: 9, Action: REBOUND, Team: Home,
Index: 10, Action: FOUL, Team: Visitor,
Index: 11, Action: FREE_THROW_ATTEMPT, Team: Home,
Index: 12, Action: FREE_THROW_ATTEMPT, Team: Home,
Index: 13, Action: REBOUND, Team: Visitor,
Index: 14, Action: FIELD_GOAL_MADE, Team: Visitor,
Index: 15, Action: FIELD_GOAL_MADE, Team: Home,
Index: 16, Action: FOUL, Team: Home,
Index: 17, Action: FIELD_GOAL_MISSED, Team: Visitor,
Index: 18, Action: REBOUND, Team: Visitor,
Index: 19, Action: FIELD_GOAL_MADE, Team: Visitor,
Index: 20, Action: TURNOVER, Team: HOME_TURNOVER
Index: 21, Action: TURNOVER, Te